Install libraries

In [1]:
!pip install sentinelsat
!pip install geopandas
!pip install rasterio

     |████████████████████████████████| 931kB 2.7MB/s 
     |████████████████████████████████| 10.4MB 7.9MB/s 
     |████████████████████████████████| 14.7MB 313kB/s 
     |████████████████████████████████| 18.1MB 1.2MB/s 


Import libraries

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
from sentinelsat import SentinelAPI
import folium
from shapely.geometry import MultiPolygon
from shapely.geometry import Polygon
from shapely.geometry import shape
from shapely.geometry import box
import urllib.request as request
import json
from datetime import datetime
import zipfile
import rasterio as rio
from rasterio.plot import show
import rasterio.mask
import os

Extract Geometries

In [0]:
max_records=10
first_record = 18290
current_record = first_record
names = []
countries = []
latitudes = []
longitudes = []
geometries = []
while len(names) < max_records:
  water_url = f'https://water.blue-dot-observatory.com/api/waterbodies/{current_record}/index.html'
  try:
    water_body_data = json.loads(request.urlopen(water_url).read().decode())
    names.append(water_body_data['properties']['name'])
    countries.append(water_body_data['properties']['country'])
    latitudes.append(water_body_data['properties']['lat'])
    longitudes.append(water_body_data['properties']['long'])
    geometries.append(water_body_data['nominal_outline']['geometry'])
  except:
    pass
  current_record += 1

Create Dataframe

In [0]:
water_bodies = pd.DataFrame({'Name':names,
                             'Country':countries,
                             'Latitude':latitudes,
                             'Longitude':longitudes,
                             'Geometry':geometries})

In [5]:
water_bodies

,Name,Country,Latitude,Longitude,Geometry
0,Lake Travis,United States of America,30.442056,-98.004388,"{'coordinates': [[[[-98.0217194, 30.3745221], ..."
1,Lake Arlington,United States of America,32.703161,-97.213846,"{'coordinates': [[[-97.2335181, 32.6921516], [..."
2,Lake Buchanan,United States of America,30.818283,-98.422734,"{'coordinates': [[[-98.4783375, 30.7971736], [..."
3,Walter E Long Lake,United States of America,30.297869,-97.601663,"{'coordinates': [[[-97.6199675, 30.303003], [-..."
4,Cooling Water Reservoir,United States of America,28.765775,-96.049176,"{'coordinates': [[[-96.080274, 28.7517011], [-..."
5,Lake Tawakoni,United States of America,32.882111,-95.991711,"{'coordinates': [[[-96.1319487, 32.8534733], [..."
6,Eagle Mountain Dam,United States of America,32.919427,-97.497142,"{'coordinates': [[[-97.529902, 32.9085778], [-..."
7,Twin Buttes,United States of America,31.358521,-100.543715,"{'coordinates': [[[[-100.6083856, 31.3296567],..."
8,Peacock Site 1-A Tailings Dam,United States of America,32.969099,-94.674863,"{'coordinates': [[[-94.6819945, 32.9681979], [..."
9,Somerville Dam,United States of America,30.311131,-96.586281,"{'coordinates': [[[[-96.5679466, 30.2964676], ..."


Plot Map

In [0]:
my_map = folium.Map(location = [30.442056, -98.004388], zoom_start=11)

In [7]:
for water_body in water_bodies['Geometry']:
  folium.GeoJson(shape(water_body)).add_to(my_map)
my_map

Read login credentials

In [0]:
with open('/content/drive/My Drive/Sentinel Hub Credentials/login_info.txt') as file:
  credentials = file.readlines()
  credentials = [credential.strip() for credential in credentials]

Download Satellite Images

In [0]:
api = SentinelAPI(credentials[0], credentials[1], 'https://scihub.copernicus.eu/dhus')

In [0]:
today = datetime.today().strftime('%Y%m%d')

In [0]:
water_body_geometry = water_bodies.iloc[0]['Geometry']

In [0]:
water_body_shape = shape(water_body_geometry)
shape_simplified = water_body_shape.simplify(0.01, preserve_topology=True)

In [20]:
results = api.query(shape_simplified, date = ('20140101', today), platformname = 'Sentinel-2', processinglevel = 'Level-1C', cloudcoverpercentage = (0,5))
results_gdf = api.to_geodataframe(results).sort_values(['cloudcoverpercentage'], ascending=True)
uuid = results_gdf.iloc[0]['uuid']
filename = '/content/' + results_gdf.iloc[0]['title'] + '.zip'
data = api.download(uuid)

Querying products: 100%|██████████| 139/139 [00:02<00:00, 57.19 products/s]
/usr/local/lib/python3.6/dist-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
Downloading: 100%|██████████| 869M/869M [01:14<00:00, 11.7MB/s]
MD5 checksumming: 100%|██████████| 869M/869M [00:02<00:00, 432MB/s]


In [0]:
with zipfile.ZipFile(filename, 'r') as zip_ref:
  zip_ref.extractall('/content/drive/My Drive/Satellite Data/')
folder = os.listdir('/content/drive/My Drive/Satellite Data/' + data['title'] + '.SAFE/GRANULE/')[0]
directory = '/content/drive/My Drive/Satellite Data/' + data['title'] + '.SAFE/GRANULE/' + folder + '/IMG_DATA/'

In [0]:
os.remove(filename)

In [0]:
for f in os.listdir(directory):
  if '_TC' in f:
    true_color = rio.open(directory + f)

In [0]:
show(true_color)